<a href="https://colab.research.google.com/github/ankit-kothari/Data-Science-Journey/blob/master/ner_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers 
!wget -nc https://lazyprogrammer.me/course_files/AirlineTweets.csv
!pip install dask
!pip install 'fsspec>=0.3.3'

In [ ]:
!pip install datasets
!pip install torchinfo

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [5]:
from transformers import pipeline
import torch
import spacy
import tqdm as notebook_tqdm
from torchinfo import summary
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score,roc_auc_score,accuracy_score
from sklearn.metrics import plot_confusion_matrix,classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from datasets import load_dataset
from spacy.lang.en import English
import warnings
from tqdm import tqdm
import re
import string
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import concurrent.futures
import dask.bag as db
import dask 
import graphviz
from dask import visualize

## Loading the Data

In [6]:
from datasets import load_dataset

In [7]:
data = load_dataset('conll2003')

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

- tokens are list of words
- words are not lower-cased and puntuations are present

In [9]:
data['train'][4]

{'chunk_tags': [11,
  11,
  12,
  13,
  11,
  12,
  12,
  11,
  12,
  12,
  12,
  12,
  21,
  13,
  11,
  12,
  21,
  22,
  11,
  13,
  11,
  1,
  13,
  11,
  17,
  11,
  12,
  12,
  21,
  1,
  0],
 'id': '4',
 'ner_tags': [5,
  0,
  0,
  0,
  0,
  3,
  4,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'pos_tags': [22,
  27,
  21,
  35,
  12,
  22,
  22,
  27,
  16,
  21,
  22,
  22,
  38,
  15,
  22,
  24,
  20,
  37,
  21,
  15,
  24,
  16,
  15,
  22,
  15,
  12,
  16,
  21,
  38,
  17,
  7],
 'tokens': ['Germany',
  "'s",
  'representative',
  'to',
  'the',
  'European',
  'Union',
  "'s",
  'veterinary',
  'committee',
  'Werner',
  'Zwingmann',
  'said',
  'on',
  'Wednesday',
  'consumers',
  'should',
  'buy',
  'sheepmeat',
  'from',
  'countries',
  'other',
  'than',
  'Britain',
  'until',
  'the',
  'scientific',
  'advice',
  'was',
  'clearer',
  '.']}

In [10]:
data['train'][4]['tokens']

['Germany',
 "'s",
 'representative',
 'to',
 'the',
 'European',
 'Union',
 "'s",
 'veterinary',
 'committee',
 'Werner',
 'Zwingmann',
 'said',
 'on',
 'Wednesday',
 'consumers',
 'should',
 'buy',
 'sheepmeat',
 'from',
 'countries',
 'other',
 'than',
 'Britain',
 'until',
 'the',
 'scientific',
 'advice',
 'was',
 'clearer',
 '.']

In [11]:
data['train'][4]['ner_tags']

[5,
 0,
 0,
 0,
 0,
 3,
 4,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

- Sequnce Class specifies a field contains a list of features, below is the example to extract all the label names for NER TAGS

In [12]:
for key,value in data['train'].features.items():
  print(f"{key}:{value}")

id:Value(dtype='string', id=None)
tokens:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
pos_tags:Sequence(feature=ClassLabel(num_classes=47, names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)
chunk_tags:Sequence(feature=ClassLabel(num_classes=23, names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)
ner_tags:Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)


In [13]:
tags = data['train'].features['ner_tags'].feature
tags

ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)

#### Mapping the NER Integer to Classlabel in the Datadict object

In [14]:
def create_tag_name(batch):
  return {"ner_tags_str":[tags.int2str(idx) for idx in batch['ner_tags']]}

data = data.map(create_tag_name)

Parameter 'function'=<function create_tag_name at 0x7f8f0be0ab00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/14042 [00:00<?, ?ex/s]

  0%|          | 0/3251 [00:00<?, ?ex/s]

  0%|          | 0/3454 [00:00<?, ?ex/s]

In [15]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3454
    })
})

In [16]:
data['train'][4]['ner_tags_str']

['B-LOC',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

#### Checking the Distribution of Labels across Datasets

In [17]:
from collections import Counter
from collections import defaultdict

In [18]:
split2freq = defaultdict(Counter)
for split,dataset in data.items():
  for row in dataset['ner_tags_str']:
    for tag in row:
      if tag.startswith('B'):
        tag_type = tag.split('-')[1]
        split2freq[split][tag_type] +=1
  
pd.DataFrame.from_dict(split2freq,orient="index")
  

,ORG,MISC,PER,LOC
train,6321,3438,6600,7140
validation,1341,922,1842,1837
test,1661,702,1617,1668


## Tokenization

In [19]:
from transformers import AutoTokenizer
bert_model_base = 'bert-base-cased'
roberta_model_base = 'xlm-roberta-base'
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_base)
xlm_roberta_toeknizer = AutoTokenizer.from_pretrained(roberta_model_base)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [20]:
text = " ".join(data['train'][4]['tokens'])
print(text)
sample, sample_bert, sample_roberta = tokenizer(text), bert_tokenizer(text),xlm_roberta_toeknizer(text)
tokenizer_comparisons = pd.DataFrame([sample.tokens(),sample_bert.tokens(),sample_roberta.tokens()],index=['distillBERT','BERT','RoBERTa'])
tokenizer_comparisons

Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
distillBERT,[CLS],Germany,',s,representative,to,the,European,Union,',s,veterinary,committee,Werner,Z,##wing,##mann,said,on,Wednesday,consumers,should,buy,sheep,##me,##at,from,countries,other,than,Britain,until,the,scientific,advice,was,clearer,.,[SEP],None,None,None,None,None,None
BERT,[CLS],Germany,',s,representative,to,the,European,Union,',s,veterinary,committee,Werner,Z,##wing,##mann,said,on,Wednesday,consumers,should,buy,sheep,##me,##at,from,countries,other,than,Britain,until,the,scientific,advice,was,clearer,.,[SEP],None,None,None,None,None,None
RoBERTa,<s>,▁Germany,▁',s,▁representativ,e,▁to,▁the,▁European,▁Union,▁',s,▁veterinar,y,▁committee,▁Werner,▁Z,wing,mann,▁said,▁on,▁Wednesday,▁consumer,s,▁should,▁buy,▁she,ep,me,at,▁from,▁countries,▁other,▁than,▁Britain,▁until,▁the,▁scientific,▁advice,▁was,▁clear,er,▁,.,</s>


- Input Sequence could be longer than the target sequnce given the word is splitup into subwords.

In [21]:
print(type(sample))
print(f" length of Input Tokens:{len(sample.tokens())} and {sample.tokens()}")
print(f" word_ids {sample.word_ids()}")
print(f" length of Output Tokens:{len(data['train'][0]['ner_tags_str'])} and {data['train'][0]['ner_tags_str']}")

<class 'transformers.tokenization_utils_base.BatchEncoding'>
 length of Input Tokens:39 and ['[CLS]', 'Germany', "'", 's', 'representative', 'to', 'the', 'European', 'Union', "'", 's', 'veterinary', 'committee', 'Werner', 'Z', '##wing', '##mann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']
 word_ids [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17, 18, 19, 20, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, None]
 length of Output Tokens:9 and ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


#### Target Alignment

- For any word split into multiple tokens, we assign the same target
- "-100" should be assigned to tokens which the model should ignore for example [CLS] and [SEP] tokens. 
- We need encoding for these becasue many to many tasks each input produces an output

In [55]:
sample_tokens = data['train'][4]['tokens']
sample_ner_ids = data['train'][4]['ner_tags']
sample_labels = data['train'][4]['ner_tags_str']
sample_str = tokenizer(sample_tokens, is_split_into_words=True)
print(sample_str)
sample_tokens = sample_str.tokens() #convert input_ids to tokens 
sample_ids = sample_str.word_ids() #thats the total number of labels we need or present in the input id.

{'input_ids': [101, 1860, 112, 188, 4702, 1106, 1103, 1735, 1913, 112, 188, 27431, 3914, 14651, 163, 7635, 4119, 1163, 1113, 9031, 11060, 1431, 4417, 8892, 3263, 2980, 1121, 2182, 1168, 1190, 2855, 1235, 1103, 3812, 5566, 1108, 27830, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [65]:
#misalignment of labels and ids 
print(f"count of token ids after tokenization {len(sample_ids)}") #sample ids after tokenization 
print(f"count of labels by each word {len(sample_labels)}") #sample labels by words

count of token ids after tokenization 39
count of labels by each word 31


In [66]:
#fixing the misallignment
new_tag = []
new_ids =[]
for idx in sample_ids:
  if idx==None:
    new_tag.append(-100)
    new_ids.append(-100)
  else:
    new_tag.append(sample_labels[idx])
    new_ids.append(sample_ner_ids[idx])

In [67]:
  
  pd.DataFrame([sample_tokens,sample_ids,new_ids,new_tag],index=["tokens","ids","ner_ids","new_tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
tokens,[CLS],Germany,',s,representative,to,the,European,Union,',s,veterinary,committee,Werner,Z,##wing,##mann,said,on,Wednesday,consumers,should,buy,sheep,##me,##at,from,countries,other,than,Britain,until,the,scientific,advice,was,clearer,.,[SEP]
ids,None,0,1,1,2,3,4,5,6,7,7,8,9,10,11,11,11,12,13,14,15,16,17,18,18,18,19,20,21,22,23,24,25,26,27,28,29,30,None
ner_ids,-100,5,0,0,0,0,0,3,4,0,0,0,0,1,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,-100
new_tags,-100,B-LOC,O,O,O,O,O,B-ORG,I-ORG,O,O,O,O,B-PER,I-PER,I-PER,I-PER,O,O,O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,-100


In [61]:
len(sample_labels)

31

In [68]:
tag_names = data['train'].features['ner_tags'].feature.names
idx2tag = {idx:tag for idx, tag in enumerate(tag_names)}
tag2idx = {tag:idx for idx, tag in enumerate(tag_names)}


#### Tokenizing the entire Dataset

In [269]:
def tokenize_fn(batch):
  tokenized_inputs= tokenizer(batch['tokens'], truncation=True, is_split_into_words=True) #no padding option since it will be handled by the trainer in thsi case
  alligned_batch=[]
  allien_tokens=[]
  for ids,row in enumerate(batch['ner_tags']):
    ner_token = []
    ner_ids =[]
    word_ids = tokenized_inputs.word_ids(ids)
    token_ids = tokenized_inputs.tokens(ids)
    for word_id  in word_ids:
      if word_id==None:
        ner_ids.append(-100)
      else:
        ner_ids.append(row[word_id])
    alligned_batch.append(ner_ids)
    allien_tokens.append(token_ids)
  tokenized_inputs['labels']=alligned_batch
  #tokenized_inputs['tokenized_tokens']=allien_tokens
  return tokenized_inputs




In [270]:
%%time
tokenized_dataset = data.map(tokenize_fn, batched=True, remove_columns=data['train'].column_names)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

CPU times: user 5.19 s, sys: 54.7 ms, total: 5.25 s
Wall time: 4.16 s


In [271]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3454
    })
})

In [272]:
len(tokenized_dataset['train'][4]['input_ids'])


39

In [273]:
tokenizer.decode(tokenized_dataset['train'][4]['input_ids'])

"[CLS] Germany's representative to the European Union's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer. [SEP]"

In [274]:
#len(tokenized_dataset['train'][4]['tokenized_tokens'])

In [275]:
#tokenized_tokens = tokenized_dataset['train'][4]['tokenized_tokens']
tokenized_input_ids = tokenized_dataset['train'][4]['input_ids']
tokenized_labels = tokenized_dataset['train'][4]['labels']
tokenized_tags =[None if label == -100 else idx2tag[label] for label in tokenized_dataset['train'][4]['labels'] ]

In [276]:
#tokenized_sample= pd.DataFrame([tokenized_tokens,tokenized_input_ids,tokenized_labels,tokenized_tags],index=['tokens','input_ids','alligned_labels','alligned_tags'])


In [277]:
#tokenized_sample

In [278]:
#column names to remove which are not needed
data['train'].column_names
# can pass remove_columns in the map function

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str']

#### Data Collator Object

In [279]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [280]:
#one dictionary with all the samples in one key in the tokenized dataset
tokenized_dataset['train'][0:2]

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

In [281]:
type(tokenized_dataset['train'][4]['labels'][0])

int

In [282]:
#returning individual dictionary which can be passed to the Data Collator object
tokenized_sample_list = [tokenized_dataset['train'][i] for i in range(2)]

In [283]:
batch = data_collator(tokenized_sample_list)
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])